In [ ]:
import nipype
import nipype.interfaces.utility as niu
import nipype.interfaces.io as nio
from nipype.interfaces import fsl, freesurfer, ants
from nipype.pipeline import engine as pe
from nipype.interfaces.image import Reorient

import os

In [ ]:
subject_ids = [str(x).zfill(2) for x in range(1,20)]
subject_ids

## To do next:
1. Warp all masks to T1w space, then to MNI space
2. PCA in MNI space
3. Warp back to individual space
4. Re-create subsegments in individual (T1w) space
5. Warp subsegments to FLASH space for visualization

### DS-01: Affine and SyN warp

In [5]:
project_folder = '/home/stevenm/Projects/bias'
workflow = pe.Workflow(name='warp_masks_to_t1w_and_mni_ds01')
workflow.base_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders')
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# identity
identity = pe.Node(niu.IdentityInterface(fields=['subject_id', 'hemisphere']), name='identity')
identity.iterables = [('subject_id', subject_ids),
                      ('hemisphere', ['l', 'r'])]

# selector
templates = {'t1w': os.path.join(project_folder, 'derivatives', 'ds-01', 'fmriprep', 'sub-{subject_id}',  'anat',  'sub-{subject_id}_desc-preproc_T1w.nii.gz'),
             'stn_mask': os.path.join(project_folder, 'derivatives', 'ds-01', 'conjunct_masks', 'sub-{subject_id}', 'anat', 'sub-{subject_id}_space-FLASH_desc-stn{hemisphere}_mask.nii.gz'),
             # 'flash_to_t1w': os.path.join(project_folder, 'derivatives', 'ds-01', 'coregister_flash_to_t1w', 'forward_xfm', '_subject_id_{subject_id}', 'fixed_affine_bbreg_sub-{subject_id}.mat'),
             'flash_to_t1w': os.path.join(project_folder, 'derivatives', 'ds-01', 'coregister_flash_to_t1w', 'sub-{subject_id}', 'sub-{subject_id}_from-FLASH_to-T1w_mode-image_xfm.txt'),
             't1w_to_mni': os.path.join(project_folder, 'derivatives', 'ds-01', 'fmriprep', 'sub-{subject_id}', 'anat', 'sub-{subject_id}_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5')
             }

selector = pe.Node(nio.SelectFiles(templates), name='selector')

workflow.connect(identity, 'subject_id', selector, 'subject_id')
workflow.connect(identity, 'hemisphere', selector, 'hemisphere')


### First warp
apply_xfm = pe.Node(ants.ApplyTransforms(), name='applyxfm_flash_2_t1w')
apply_xfm.inputs.interpolation = 'NearestNeighbor'
workflow.connect(selector, 'stn_mask', apply_xfm, 'input_image')
workflow.connect(selector, 'flash_to_t1w', apply_xfm, 'transforms')
workflow.connect(selector, 't1w', apply_xfm, 'reference_image')

### First and second warp in one go. Merge warps
merger = pe.Node(niu.Merge(2), 'warp_merger')
workflow.connect(selector, 'flash_to_t1w', merger, 'in2')  # note that transforms are inputted in 'reverse order'
workflow.connect(selector, 't1w_to_mni', merger, 'in1')

apply_syn = pe.Node(ants.ApplyTransforms(), name='applysyn_t1w_to_mni')
apply_syn.inputs.reference_image = '/home/stevenm/Templates/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii'
apply_syn.inputs.interpolation = 'NearestNeighbor'
workflow.connect(merger, 'out', apply_syn, 'transforms')
workflow.connect(selector, 'stn_mask', apply_syn, 'input_image')


## figure
# reporter = pe.Node(niu.Function(input_names=['fixed_image', 'moving_image'], output_names=['out_report'], function=make_report), name='reporter')
# # workflow.connect(selector, 't1w', reporter, 'fixed_image')
# reporter.inputs.fixed_image = '/home/stevenm/Templates/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii'
# workflow.connect(apply_syn, 'output_image', reporter, 'moving_image')


##
ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = os.path.join(project_folder, 'derivatives', 'ds-01', 'masks_warped')
ds.inputs.regexp_substitutions = [('t1w/_hemisphere_([a-z])_subject_id_([0-9]{2})/sub-([0-9]{2})_space-FLASH_desc-stn[a-z]_mask_trans.nii.gz', 'sub-\\2/sub-\\2_space-T1w_desc-stn\\1_mask.nii.gz'),
                                  ('mni/_hemisphere_([a-z])_subject_id_([0-9]{2})/sub-([0-9]{2})_space-FLASH_desc-stn[a-z]_mask_trans.nii.gz', 'sub-\\2/sub-\\2_space-MNI152NLin2009cAsym_desc-stn\\1_mask.nii.gz'),
                                  ]

workflow.connect(apply_xfm, 'output_image', ds, 't1w')
workflow.connect(apply_syn, 'output_image', ds, 'mni')
# workflow.connect(reporter, 'out_report', ds, 'registration_reports_masks_in_mni')

In [ ]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 10, 'memory_gb': 250})    # should be fast

220304-09:59:56,987 nipype.workflow INFO:
	 Workflow warp_masks_to_t1w_and_mni_ds01 settings: ['check', 'execution', 'logging', 'monitoring']
220304-09:59:57,183 nipype.workflow INFO:
	 Running in parallel.
220304-09:59:57,190 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 38 jobs ready. Free memory (GB): 250.00/250.00, Free processors: 10/10.
220304-09:59:57,295 nipype.workflow INFO:
	 [Node] Setting-up "warp_masks_to_t1w_and_mni_ds01.selector" in "/home/stevenm/Projects/bias/processing/nipype_workflow_folders/warp_masks_to_t1w_and_mni_ds01/_hemisphere_r_subject_id_19/selector".
220304-09:59:57,300 nipype.workflow INFO:
	 [Node] Setting-up "warp_masks_to_t1w_and_mni_ds01.selector" in "/home/stevenm/Projects/bias/processing/nipype_workflow_folders/warp_masks_to_t1w_and_mni_ds01/_hemisphere_l_subject_id_17/selector".
220304-09:59:57,301 nipype.workflow INFO:
	 [Node] Running "selector" ("nipype.interfaces.io.SelectFiles")
220304-09:59:57,301 nipype.workflow INFO:
	 [Node] Sett

### DS-02: Only SyN warp

In [4]:
import glob
subject_ids = [x.split('-')[-1].split('.')[0] for x in sorted(glob.glob('./derivatives/ds-02/fmriprep/sub*.html'))]
subject_ids

['01',
 '02',
 '03',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15']

In [9]:
project_folder = '/home/stevenm/Projects/bias'
workflow = pe.Workflow(name='warp_masks_to_t1w_and_mni_ds02')
workflow.base_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders')
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# identity
identity = pe.Node(niu.IdentityInterface(fields=['subject_id', 'hemisphere']), name='identity')
identity.iterables = [('subject_id', subject_ids),
                      ('hemisphere', ['l', 'r'])]

# selector
templates = {'t1w': os.path.join(project_folder, 'derivatives', 'ds-02', 'fmriprep', 'sub-{subject_id}',  'anat',  'sub-{subject_id}_desc-preproc_T1w.nii.gz'),
             'stn_mask': os.path.join(project_folder, 'derivatives', 'ds-02', 'conjunct_masks', 'sub-{subject_id}', 'anat', 'sub-{subject_id}_desc-stn{hemisphere}_mask.nii.gz'),
             't1w_to_mni': os.path.join(project_folder, 'derivatives', 'ds-02', 'fmriprep', 'sub-{subject_id}', 'anat', 'sub-{subject_id}_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5')
             }

selector = pe.Node(nio.SelectFiles(templates), name='selector')

workflow.connect(identity, 'subject_id', selector, 'subject_id')
workflow.connect(identity, 'hemisphere', selector, 'hemisphere')


### First warp
# def apply_affine(source, target):
#     import nighres
#     import os
#     out = nighres.registration.embedded_antsreg(source, target, run_rigid=False, run_affine=False, run_syn=False, save_data=True, output_dir='./')
    
#     return os.path.abspath(out['transformed_source'])
# affine_fixer = pe.Node(niu.Function(input_names=['source', 'target'], output_names=['out_file'], function=apply_affine), 'affine_fixer')
# workflow.connect(selector, 'stn_mask', affine_fixer, 'source')
# workflow.connect(selector, 't1w', affine_fixer, 'target')

## For some annoying reason, the masks are in LAS, whereas the original t1w is in PSR, and the fmriprepped data is in RAS...
reorienter = pe.Node(Reorient(orientation='RAS'), 'reorienter')
workflow.connect(selector, 'stn_mask', reorienter, 'in_file')


apply_xfm = pe.Node(ants.ApplyTransforms(), name='applyxfm_t1w_2_mni')
apply_xfm.inputs.interpolation = 'NearestNeighbor'
apply_xfm.inputs.reference_image = '/home/stevenm/Templates/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii'
workflow.connect(reorienter, 'out_file', apply_xfm, 'input_image')
workflow.connect(selector, 't1w_to_mni', apply_xfm, 'transforms')

# workflow.connect(selector, 't1w', apply_xfm, 'reference_image')

### First and second warp in one go. Merge warps
# merger = pe.Node(niu.Merge(2), 'warp_merger')
# workflow.connect(selector, 'flash_to_t1w', merger, 'in2')  # note that transforms are inputted in 'reverse order'
# workflow.connect(selector, 't1w_to_mni', merger, 'in1')

# apply_syn = pe.Node(ants.ApplyTransforms(), name='applysyn_t1w_to_mni')
# apply_syn.inputs.reference_image = '/home/stevenm/Templates/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii'
# apply_syn.inputs.interpolation = 'NearestNeighbor'
# workflow.connect(merger, 'out', apply_syn, 'transforms')
# workflow.connect(selector, 'stn_mask', apply_syn, 'input_image')


## figure
# reporter = pe.Node(niu.Function(input_names=['fixed_image', 'moving_image'], output_names=['out_report'], function=make_report), name='reporter')
# # workflow.connect(selector, 't1w', reporter, 'fixed_image')
# reporter.inputs.fixed_image = '/home/stevenm/Templates/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii'
# workflow.connect(apply_syn, 'output_image', reporter, 'moving_image')


##
ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = os.path.join(project_folder, 'derivatives', 'ds-02', 'masks_warped') # sub-01_desc-stnl_mask_ants-def0_trans.nii.gz
# ds.inputs.regexp_substitutions = [('mni/_hemisphere_([a-z])_subject_id_([0-9]{2})/sub-([0-9]{2})_desc-stn[a-z]_mask_ants-def0_trans.nii.gz', 'sub-\\2/sub-\\2_space-MNI152NLin2009cAsym_desc-stn\\1_mask.nii.gz'),
#                                   ('affine_corrected_masks/_hemisphere_([a-z])_subject_id_([0-9]{2})/sub-([0-9]{2})_desc-stn[a-z]_mask_ants-def0.nii.gz', 'sub-\\2/sub-\\2_space-T1w-RAS_desc-stn\\1_mask.nii.gz')
#                                  ]
ds.inputs.regexp_substitutions = [('mni/_hemisphere_([a-z])_subject_id_([0-9]{2})/sub-([0-9]{2})_desc-stn[a-z]_mask_ras_trans.nii.gz', 'sub-\\2/sub-\\2_space-MNI152NLin2009cAsym_desc-stn\\1_mask.nii.gz'),
                                  ('affine_corrected_masks/_hemisphere_([a-z])_subject_id_([0-9]{2})/sub-([0-9]{2})_desc-stn[a-z]_mask_ras.nii.gz', 'sub-\\2/sub-\\2_space-T1w-RAS_desc-stn\\1_mask.nii.gz')
                                 ]

workflow.connect(apply_xfm, 'output_image', ds, 'mni')
workflow.connect(reorienter, 'out_file', ds, 'affine_corrected_masks')

# workflow.connect(reporter, 'out_report', ds, 'registration_report')

In [ ]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 10, 'memory_gb': 250})    # should be fast

220304-19:55:53,587 nipype.workflow INFO:
	 Workflow warp_masks_to_t1w_and_mni_ds02 settings: ['check', 'execution', 'logging', 'monitoring']
220304-19:55:53,655 nipype.workflow INFO:
	 Running in parallel.
220304-19:55:53,659 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 28 jobs ready. Free memory (GB): 250.00/250.00, Free processors: 10/10.
220304-19:55:53,748 nipype.workflow INFO:
	 [Node] Setting-up "warp_masks_to_t1w_and_mni_ds02.selector" in "/home/stevenm/Projects/bias/processing/nipype_workflow_folders/warp_masks_to_t1w_and_mni_ds02/_hemisphere_r_subject_id_15/selector".
220304-19:55:53,750 nipype.workflow INFO:
	 [Node] Setting-up "warp_masks_to_t1w_and_mni_ds02.selector" in "/home/stevenm/Projects/bias/processing/nipype_workflow_folders/warp_masks_to_t1w_and_mni_ds02/_hemisphere_l_subject_id_15/selector".
220304-19:55:53,751 nipype.workflow INFO:
	 [Node] Setting-up "warp_masks_to_t1w_and_mni_ds02.selector" in "/home/stevenm/Projects/bias/processing/nipype_workflow

## Finally, warp Accolla masks

#### DS-01: warp both to T1w and FLASH space

In [12]:
subject_ids = [str(x).zfill(2) for x in range(1,20)]
subject_ids

['01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19']

In [13]:
project_folder = '/home/stevenm/Projects/bias'
workflow = pe.Workflow(name='warp_accolla_masks_to_t1w_ds01')
workflow.base_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders')
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# identity
identity = pe.Node(niu.IdentityInterface(fields=['subject_id', 'subroi']), name='identity')
identity.iterables = [('subject_id', subject_ids),
                      ('subroi', ['associative', 'limbic', 'motor'])]

# selector
templates = {'t1w': os.path.join(project_folder, 'derivatives', 'ds-01', 'fmriprep', 'sub-{subject_id}',  'anat',  'sub-{subject_id}_desc-preproc_T1w.nii.gz'),
             'stn_mask': os.path.join(project_folder, 'sourcedata', 'Accolla_atlas', 'mixed', 'STN_{subroi}.nii'),
             'mni_to_t1w': os.path.join(project_folder, 'derivatives', 'ds-01', 'fmriprep', 'sub-{subject_id}', 'anat', 'sub-{subject_id}_from-MNI152NLin2009cAsym_to-T1w_mode-image_xfm.h5'),
             't1w_to_flash': os.path.join(project_folder, 'derivatives', 'ds-01', 'coregister_flash_to_t1w', 'sub-{subject_id}', 'sub-{subject_id}_from-T1w_to-FLASH_mode-image_xfm.txt'),
             'flash_echo1': os.path.join(project_folder, 'sourcedata', 'sourcedata', 'ds-01', 'sub-{subject_id}', 'anat', 'sub-{subject_id}_echo-1_FLASH.nii.gz'),
            }

selector = pe.Node(nio.SelectFiles(templates), name='selector')

workflow.connect(identity, 'subject_id', selector, 'subject_id')
workflow.connect(identity, 'subroi', selector, 'subroi')


def split_left_right(mask_fn):
    from nipype.utils.filemanip import split_filename
    import nibabel as nib
    import os.path as op
    _, fn, ext = split_filename(mask_fn)

    nii = nib.load(mask_fn)
    
    midline = int(nii.shape[0]/2)
    data = nii.get_fdata().copy()
    data_left = data.copy()
    data_left[midline:,:,:] = 0
    data_right = data.copy()
    data_right[:midline,:,:] = 0
    
    ## also make gz
    # force to integers - mask should be bool
    if mask_fn.endswith('.nii'):
        nib.Nifti1Image(nii.get_fdata().round().astype(int), header=nii.header, affine=nii.affine).to_filename(mask_fn + '.gz')
    
    left_fn = op.abspath('{}_left.nii.gz'.format(fn))
    right_fn = op.abspath('{}_right.nii.gz'.format(fn))
    
    nib.Nifti1Image(data_left.round().astype(int), header=nii.header, affine=nii.affine).to_filename(left_fn)
    nib.Nifti1Image(data_right.round().astype(int), header=nii.header, affine=nii.affine).to_filename(right_fn)
    return [mask_fn+'.gz', left_fn, right_fn]

splitter = pe.Node(niu.Function(input_names=['mask_fn'], output_names=['splitted'], function=split_left_right), 'splitter')
workflow.connect(selector, 'stn_mask', splitter, 'mask_fn')


### First warp
apply_xfm = pe.MapNode(ants.ApplyTransforms(), name='applyxfm_mni_2_t1w', iterfield='input_image')
apply_xfm.inputs.interpolation = 'NearestNeighbor'
workflow.connect(splitter, 'splitted', apply_xfm, 'input_image')
workflow.connect(selector, 'mni_to_t1w', apply_xfm, 'transforms')
workflow.connect(selector, 't1w', apply_xfm, 'reference_image')


## First and second warp in one go. Merge warps
merger = pe.Node(niu.Merge(2), 'warp_merger')
workflow.connect(selector, 'mni_to_t1w', merger, 'in2')  # note that transforms are inputted in 'reverse order'
workflow.connect(selector, 't1w_to_flash', merger, 'in1')

apply_syn = pe.MapNode(ants.ApplyTransforms(), name='applysyn_mni_2_flash', iterfield='input_image')
apply_syn.inputs.interpolation = 'NearestNeighbor'
workflow.connect(merger, 'out', apply_syn, 'transforms')
workflow.connect(splitter, 'splitted', apply_syn, 'input_image')
workflow.connect(selector, 'flash_echo1', apply_syn, 'reference_image')


##
ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = os.path.join(project_folder, 'derivatives', 'ds-01', 'masks_warped_accolla')
ds.inputs.regexp_substitutions = [(r'masks_warped_accolla/(\S+)/_subject_id_([0-9]{2})_subroi_([a-z]+)/.*/STN_([a-z]+)_trans.nii.gz', 
                                   'masks_warped_accolla/sub-\\2/sub-\\2_space-\\1_desc-stn-\\3_mask.nii.gz'),
                                  (r'masks_warped_accolla/(\S+)/_subject_id_([0-9]{2})_subroi_([a-z]+)/.*/STN_([a-z]+)_left_trans.nii.gz', 
                                   'masks_warped_accolla/sub-\\2/sub-\\2_space-\\1_desc-stnl-\\3_mask.nii.gz'),
                                  (r'masks_warped_accolla/(\S+)/_subject_id_([0-9]{2})_subroi_([a-z]+)/.*/STN_([a-z]+)_right_trans.nii.gz', 
                                   'masks_warped_accolla/sub-\\2/sub-\\2_space-\\1_desc-stnr-\\3_mask.nii.gz'),
                                  ]

workflow.connect(apply_xfm, 'output_image', ds, 'T1w')
workflow.connect(apply_syn, 'output_image', ds, 'FLASH')
# workflow.connect(reporter, 'out_report', ds, 'registration_report')

In [ ]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 20, 'memory_gb': 250})    # should be fast

220306-14:17:55,762 nipype.workflow INFO:
	 Workflow warp_accolla_masks_to_t1w_ds01 settings: ['check', 'execution', 'logging', 'monitoring']
220306-14:17:55,959 nipype.workflow INFO:
	 Running in parallel.
220306-14:17:55,966 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 57 jobs ready. Free memory (GB): 250.00/250.00, Free processors: 20/20.
220306-14:17:56,119 nipype.workflow INFO:
	 [Node] Setting-up "warp_accolla_masks_to_t1w_ds01.selector" in "/home/stevenm/Projects/bias/processing/nipype_workflow_folders/warp_accolla_masks_to_t1w_ds01/_subject_id_19_subroi_motor/selector".
220306-14:17:56,125 nipype.workflow INFO:
	 [Node] Setting-up "warp_accolla_masks_to_t1w_ds01.selector" in "/home/stevenm/Projects/bias/processing/nipype_workflow_folders/warp_accolla_masks_to_t1w_ds01/_subject_id_18_subroi_limbic/selector".
220306-14:17:56,123 nipype.workflow INFO:
	 [Node] Setting-up "warp_accolla_masks_to_t1w_ds01.selector" in "/home/stevenm/Projects/bias/processing/nipype_workflo

## DS-02
just to t1w space (MP2RAGEME = single space)

In [ ]:
import glob
subject_ids = [x.split('-')[-1].split('.')[0] for x in sorted(glob.glob('./derivatives/ds-02/fmriprep/sub*.html'))]
subject_ids

In [ ]:
project_folder = '/home/stevenm/Projects/bias'
workflow = pe.Workflow(name='warp_accolla_masks_to_t1w_ds02')
workflow.base_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders')
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# identity
identity = pe.Node(niu.IdentityInterface(fields=['subject_id', 'subroi']), name='identity')
identity.iterables = [('subject_id', subject_ids),
                      ('subroi', ['associative', 'limbic', 'motor'])]

# selector
templates = {'t1w': os.path.join(project_folder, 'derivatives', 'ds-02', 'fmriprep', 'sub-{subject_id}',  'anat',  'sub-{subject_id}_desc-preproc_T1w.nii.gz'),
             'stn_mask': os.path.join(project_folder, 'sourcedata', 'Accolla_atlas', 'mixed', 'STN_{subroi}.nii'),
             'mni_to_t1w': os.path.join(project_folder, 'derivatives', 'ds-02', 'fmriprep', 'sub-{subject_id}', 'anat', 'sub-{subject_id}_from-MNI152NLin2009cAsym_to-T1w_mode-image_xfm.h5'),
            }

selector = pe.Node(nio.SelectFiles(templates), name='selector')

workflow.connect(identity, 'subject_id', selector, 'subject_id')
workflow.connect(identity, 'subroi', selector, 'subroi')

def split_left_right(mask_fn):
    from nipype.utils.filemanip import split_filename
    import nibabel as nib
    import os.path as op
    _, fn, ext = split_filename(mask_fn)

    nii = nib.load(mask_fn)
    
    midline = int(nii.shape[0]/2)
    data = nii.get_fdata().copy()
    data_left = data.copy()
    data_left[midline:,:,:] = 0
    data_right = data.copy()
    data_right[:midline,:,:] = 0
    
    ## also make gz
    # force to integers - mask should be bool
    if mask_fn.endswith('.nii'):
        nib.Nifti1Image(nii.get_fdata().round().astype(int), header=nii.header, affine=nii.affine).to_filename(mask_fn + '.gz')
    
    left_fn = op.abspath('{}_left.nii.gz'.format(fn))
    right_fn = op.abspath('{}_right.nii.gz'.format(fn))
    
    nib.Nifti1Image(data_left.round().astype(int), header=nii.header, affine=nii.affine).to_filename(left_fn)
    nib.Nifti1Image(data_right.round().astype(int), header=nii.header, affine=nii.affine).to_filename(right_fn)
    return [mask_fn+'.gz', left_fn, right_fn]

splitter = pe.Node(niu.Function(input_names=['mask_fn'], output_names=['splitted'], function=split_left_right), 'splitter')
workflow.connect(selector, 'stn_mask', splitter, 'mask_fn')


apply_xfm = pe.MapNode(ants.ApplyTransforms(), name='applyxfm_mni_2_t1w', iterfield='input_image')
apply_xfm.inputs.interpolation = 'NearestNeighbor'
workflow.connect(splitter, 'splitted', apply_xfm, 'input_image')
workflow.connect(selector, 'mni_to_t1w', apply_xfm, 'transforms')
workflow.connect(selector, 't1w', apply_xfm, 'reference_image')
  

##
ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = os.path.join(project_folder, 'derivatives', 'ds-02', 'masks_warped_accolla')
ds.inputs.regexp_substitutions = [(r'masks_warped_accolla/(\S+)/_subject_id_([0-9]{2})_subroi_([a-z]+)/.*/STN_([a-z]+)_trans.nii.gz', 
                                   'masks_warped_accolla/sub-\\2/sub-\\2_space-\\1_desc-stn-\\3_mask.nii.gz'),
                                  (r'masks_warped_accolla/(\S+)/_subject_id_([0-9]{2})_subroi_([a-z]+)/.*/STN_([a-z]+)_left_trans.nii.gz', 
                                   'masks_warped_accolla/sub-\\2/sub-\\2_space-\\1_desc-stnl-\\3_mask.nii.gz'),
                                  (r'masks_warped_accolla/(\S+)/_subject_id_([0-9]{2})_subroi_([a-z]+)/.*/STN_([a-z]+)_right_trans.nii.gz', 
                                   'masks_warped_accolla/sub-\\2/sub-\\2_space-\\1_desc-stnr-\\3_mask.nii.gz'),
                                  ]

workflow.connect(apply_xfm, 'output_image', ds, 'T1w')
# workflow.connect(apply_syn, 'output_image', ds, 'FLASH')
# workflow.connect(reporter, 'out_report', ds, 'registration_report')

In [ ]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 20, 'memory_gb': 250})    # should be fast